# Pyspark Dataframes
### Qué vamos a ver
- PySpark Dataframe 
- Leer un dataset
- Comprobar los Datatypes de las columnas (Schema)
- Seleccionar columnas e indexarlas
- Check Describe option similar to Pandas
- Añadir columnas
- Eliminar columnas
- Renombrar Columnas

In [3]:
#pip install pyspark
#!pip install kagglehub

In [53]:
import pyspark
from pyspark.sql import SparkSession
import kagglehub
from pyspark.sql.functions import col, monotonically_increasing_id, when, lit, regexp_replace, udf
from pyspark.sql.types import FloatType, IntegerType
import random


In [61]:
spark = SparkSession.builder.appName('spark-E2E').getOrCreate()

25/01/20 13:01:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [62]:
spark

LEEMOS EL DATASET

In [35]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ramjasmaurya/top-1000-social-media-channels")

print("Path to dataset files:", path)

Path to dataset files: /Users/carlosoliver/.cache/kagglehub/datasets/ramjasmaurya/top-1000-social-media-channels/versions/13


In [36]:
df_youtube_sep = spark.read.option('header', 'true').csv('/Users/carlosoliver/.cache/kagglehub/datasets/ramjasmaurya/top-1000-social-media-channels/versions/13/social media influencers - Youtube sep-2022.csv')
df_youtube_jun = spark.read.option('header', 'true').csv('/Users/carlosoliver/.cache/kagglehub/datasets/ramjasmaurya/top-1000-social-media-channels/versions/13/social media influencers-youtube june 2022 - june 2022.csv')
df_youtube_nov = spark.read.option('header', 'true').csv('/Users/carlosoliver/.cache/kagglehub/datasets/ramjasmaurya/top-1000-social-media-channels/versions/13/social media influencers-youtube - --nov 2022.csv')
df_youtube_dic = spark.read.option('header', 'true').csv('/Users/carlosoliver/.cache/kagglehub/datasets/ramjasmaurya/top-1000-social-media-channels/versions/13/social media influencers-YOUTUBE - --DEC 2022.csv')

df_youtube_sep.show()
df_youtube_nov.show()

+--------------------+--------------------+------------------+-------------+------------+-------------+----------+
|                S.no|                Name|          Youtuber|      Country| Subscribers|   Category_2|Avg. views|
+--------------------+--------------------+------------------+-------------+------------+-------------+----------+
|,Avg. likes,Avg C...|                NULL|              NULL|         NULL|        NULL|         NULL|      NULL|
|                   1|            T-Series|           tseries|        India|      224.4M|Music & Dance|    141.5K|
|                   2|Cocomelon - Nurse...|         checkgate|         NULL|      142.5M|    Education|     12.5M|
|                   3|           PewDiePie|         PewDiePie|United States|      111.6M|    Animation|      1.9M|
|                   4|             MrBeast|       MrBeast6000|United States|      104.1M|  Video games|     44.8M|
|                   5|   ✿ Kids Diana Show| ✿ Kids Diana Show|         NULL|    

COMPROBAMOS EL TIPO DE DATOS QUE TENEMOS

In [37]:
df_youtube_jun.printSchema()
df_youtube_sep.printSchema()
df_youtube_nov.printSchema()
df_youtube_dic.printSchema()


root
 |-- channel name: string (nullable = true)
 |-- youTuber: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Category_2: string (nullable = true)
 |-- Subscribers count: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Views avg.: string (nullable = true)
 |-- Likes avg: string (nullable = true)
 |-- Comments avg.: string (nullable = true)

root
 |-- S.no: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Youtuber: string (nullable = true)
 |-- Country: string (nullable = true)
 |--  Subscribers: string (nullable = true)
 |-- Category_2: string (nullable = true)
 |-- Avg. views: string (nullable = true)

root
 |-- s.no: string (nullable = true)
 |-- Youtube channel: string (nullable = true)
 |-- youtuber name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Followers: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ": string (nullable = true)

root
 |-- s.no: string (nullable = true)

In [38]:
print(f"Filas: {df_youtube_dic.count()}, Columnas: {len(df_youtube_dic.columns)}")
print(f"Filas: {df_youtube_jun.count()}, Columnas: {len(df_youtube_jun.columns)}")

Filas: 1004, Columnas: 7
Filas: 1021, Columnas: 9


In [39]:
# Eliminar las columnas 'Comments avg.' y 'Likes avg'
df_youtube_jun = df_youtube_jun.drop("Comments avg.", "Likes avg", "Category_2")
df_youtube_sep = df_youtube_sep.drop("S.No")
df_youtube_nov = df_youtube_nov.drop("s.No")
df_youtube_dic = df_youtube_dic.drop("s.no")


In [55]:
df_youtube_jun.printSchema()
df_youtube_sep.printSchema()
df_youtube_nov.printSchema()
df_youtube_dic.printSchema()

root
 |-- Channel_name_jun: string (nullable = true)
 |-- Youtuber_name_jun: string (nullable = true)
 |-- Category_jun: string (nullable = true)
 |-- Subscribers_jun: integer (nullable = true)
 |-- Country_jun: string (nullable = true)
 |-- Views_avg_jun: integer (nullable = true)

root
 |-- Channel_name_sep: string (nullable = true)
 |-- Youtuber_name_sep: string (nullable = true)
 |-- Country_sep: string (nullable = true)
 |-- Subscribers_sep: integer (nullable = true)
 |-- Category_sep: string (nullable = true)
 |-- Views_avg_sep: integer (nullable = true)

root
 |-- Channel_name_nov: string (nullable = true)
 |-- Youtuber_name_nov: string (nullable = true)
 |-- Category_nov: string (nullable = true)
 |-- Subscribers_nov: integer (nullable = true)
 |-- Country_nov: string (nullable = true)
 |-- Views_avg_nov: integer (nullable = true)

root
 |-- Channel_name_dic: string (nullable = true)
 |-- Youtuber_name_dic: string (nullable = true)
 |-- Category_dic: string (nullable = true)
 |

In [47]:
df_youtube_jun = df_youtube_jun.withColumnRenamed("channel name","Channel_name_jun") \
       .withColumnRenamed("Views avg.", "Views_avg_jun")\
       .withColumnRenamed("youTuber", "Youtuber_name_jun")\
       .withColumnRenamed("Category", "Category_jun")\
       .withColumnRenamed("Subscribers count", "Subscribers_jun")\
       .withColumnRenamed("Country", "Country_jun")

df_youtube_jun.printSchema()

df_youtube_sep = df_youtube_sep.withColumnRenamed("Name","Channel_name_sep") \
       .withColumnRenamed("Avg. views", "Views_avg_sep")\
       .withColumnRenamed("Youtuber", "Youtuber_name_sep")\
       .withColumnRenamed("Category_2", "Category_sep")\
       .withColumnRenamed(" Subscribers", "Subscribers_sep")\
       .withColumnRenamed("Country", "Country_sep")

df_youtube_sep.printSchema()

df_youtube_nov = df_youtube_nov.withColumnRenamed("Youtube channel","Channel_name_nov") \
       .withColumnRenamed('Views_avg_sep', "Views_avg_nov")\
       .withColumnRenamed("youtuber name", "Youtuber_name_nov")\
       .withColumnRenamed("Category", "Category_nov")\
       .withColumnRenamed("Followers", "Subscribers_nov")\
       .withColumnRenamed("Country", "Country_nov")

df_youtube_nov.printSchema()

df_youtube_dic = df_youtube_dic.withColumnRenamed("Youtube channel","Channel_name_dic") \
       .withColumnRenamed('"', "Views_avg_dic")\
       .withColumnRenamed("youtuber name", "Youtuber_name_dic")\
       .withColumnRenamed("Category", "Category_dic")\
       .withColumnRenamed("Followers", "Subscribers_dic")\
       .withColumnRenamed("Country", "Country_dic")

df_youtube_dic.printSchema()

root
 |-- Channel_name_jun: string (nullable = true)
 |-- Youtuber_name_jun: string (nullable = true)
 |-- Category_jun: string (nullable = true)
 |-- Subscribers_jun: string (nullable = true)
 |-- Country_jun: string (nullable = true)
 |-- Views_avg_jun: string (nullable = true)

root
 |-- Channel_name_sep: string (nullable = true)
 |-- Youtuber_name_sep: string (nullable = true)
 |-- Country_sep: string (nullable = true)
 |-- Subscribers_sep: string (nullable = true)
 |-- Category_sep: string (nullable = true)
 |-- Views_avg_sep: string (nullable = true)

root
 |-- Channel_name_nov: string (nullable = true)
 |-- Youtuber_name_nov: string (nullable = true)
 |-- Category_nov: string (nullable = true)
 |-- Subscribers_nov: string (nullable = true)
 |-- Country_nov: string (nullable = true)
 |-- Views_avg_nov: string (nullable = true)

root
 |-- Channel_name_dic: string (nullable = true)
 |-- Youtuber_name_dic: string (nullable = true)
 |-- Category_dic: string (nullable = true)
 |-- Sub

In [48]:
# from pyspark.sql.functions import col, udf
# from pyspark.sql.types import IntegerType
# import random

# # Función para generar un número aleatorio en el rango dado
# def random_value():
#     return random.randint(900000, 50000000)

# # Registrar la función como UDF
# random_udf = udf(random_value, IntegerType())

# # Reemplazar las columnas con valores aleatorios
# df_youtube_jun = (
#     df_youtube_jun.withColumn("Subscribers_jun", random_udf())
#       .withColumn("Views_avg_jun", random_udf())
# )

# # Reemplazar las columnas con valores aleatorios
# df_youtube_sep = (
#     df_youtube_sep.withColumn("Subscribers_sep", random_udf())
#       .withColumn("Views_avg_sep", random_udf())
# )

# # Reemplazar las columnas con valores aleatorios
# df_youtube_nov = (
#     df_youtube_nov.withColumn("Subscribers_nov", random_udf())
#       .withColumn("Views_avg_nov", random_udf())
# )

# # Reemplazar las columnas con valores aleatorios
# df_youtube_dic = (
#     df_youtube_dic.withColumn("Subscribers_dic", random_udf())
#       .withColumn("Views_avg_dic", random_udf())
# )

In [63]:
# June

df_youtube_jun.show()

+--------------------+--------------------+-------------+---------------+-----------+-------------+
|    Channel_name_jun|   Youtuber_name_jun| Category_jun|Subscribers_jun|Country_jun|Views_avg_jun|
+--------------------+--------------------+-------------+---------------+-----------+-------------+
|             juptube|  Galinha Pintadinha|Music & Dance|       40518707|          -|     40518707|
|Baby Big Mouth Ki...|Baby Big Mouth Ki...|    Animation|       34994344|          -|     34994344|
|     shfa show India|     shfa show India|  Daily vlogs|       46142247|          -|     46142247|
|Blippi - Educatio...|Blippi - Educatio...|    Education|       44326933|          -|     44326933|
|      infobellstamil|   infobells - Tamil|    Education|       41441982|          -|     41441982|
|  Diana and Roma ARA|  Diana and Roma ARA|         NULL|       28975917|          -|     28975917|
|PeppaLatinoAmericano|Peppa Pig Español...|         Toys|       38369413|          -|     38369413|


In [64]:
df_youtube_sep.show()   

+--------------------+------------------+-------------+---------------+-------------+-------------+
|    Channel_name_sep| Youtuber_name_sep|  Country_sep|Subscribers_sep| Category_sep|Views_avg_sep|
+--------------------+------------------+-------------+---------------+-------------+-------------+
|                NULL|              NULL|         NULL|       21144020|         NULL|     21144020|
|            T-Series|           tseries|        India|       30769123|Music & Dance|     30769123|
|Cocomelon - Nurse...|         checkgate|         NULL|       11363223|    Education|     11363223|
|           PewDiePie|         PewDiePie|United States|       16878935|    Animation|     16878935|
|             MrBeast|       MrBeast6000|United States|       28450859|  Video games|     28450859|
|   ✿ Kids Diana Show| ✿ Kids Diana Show|         NULL|       17355037|    Animation|     17355037|
|         Like Nastya|       Like Nastya|         NULL|       33243296|         Toys|     33243296|


In [65]:
df_youtube_nov.show()

+------------------+--------------------+-------------+---------------+-------------+-------------+
|  Channel_name_nov|   Youtuber_name_nov| Category_nov|Subscribers_nov|  Country_nov|Views_avg_nov|
+------------------+--------------------+-------------+---------------+-------------+-------------+
|      Likes (Avg.)|     Comments (Avg.)|   Category-2|       45825655|         NULL|     45825655|
|           tseries|            T-Series|Music & Dance|       29989746|        India|     29989746|
|         checkgate|Cocomelon - Nurse...|    Education|        2218241|         NULL|      2218241|
|         PewDiePie|           PewDiePie|    Animation|       23511058|United States|     23511058|
|       MrBeast6000|             MrBeast|  Video games|        9037400|United States|      9037400|
| ✿ Kids Diana Show|   ✿ Kids Diana Show|    Animation|       46556980|         NULL|     46556980|
|       Like Nastya|         Like Nastya|         Toys|       48947871|         NULL|     48947871|


In [66]:
df_youtube_dic.show()

+------------------+--------------------+-------------+---------------+-------------+-------------+
|  Channel_name_dic|   Youtuber_name_dic| Category_dic|Subscribers_dic|  Country_dic|Views_avg_dic|
+------------------+--------------------+-------------+---------------+-------------+-------------+
|      Likes (Avg.)|     Comments (Avg.)|   Category-2|        8041578|         NULL|      8041578|
|           tseries|            T-Series|Music & Dance|       47659828|        India|     47659828|
|          setindia|           SET India|         NULL|       45251029|        India|     45251029|
|         checkgate|Cocomelon - Nurse...|    Education|        8305276|         NULL|      8305276|
|           MrBeast|             MrBeast|  Video games|       16559661|United States|     16559661|
|         PewDiePie|           PewDiePie|    Animation|       49661354|United States|     49661354|
| ✿ Kids Diana Show|   ✿ Kids Diana Show|    Animation|        3784614|         NULL|      3784614|


In [69]:
df_youtube_jun.printSchema()
df_youtube_sep.printSchema()
df_youtube_nov.printSchema()
df_youtube_dic.printSchema()

root
 |-- Channel_name_jun: string (nullable = true)
 |-- Youtuber_name_jun: string (nullable = true)
 |-- Category_jun: string (nullable = true)
 |-- Subscribers_jun: integer (nullable = true)
 |-- Country_jun: string (nullable = true)
 |-- Views_avg_jun: integer (nullable = true)

root
 |-- Channel_name_sep: string (nullable = true)
 |-- Youtuber_name_sep: string (nullable = true)
 |-- Country_sep: string (nullable = true)
 |-- Subscribers_sep: integer (nullable = true)
 |-- Category_sep: string (nullable = true)
 |-- Views_avg_sep: integer (nullable = true)

root
 |-- Channel_name_nov: string (nullable = true)
 |-- Youtuber_name_nov: string (nullable = true)
 |-- Category_nov: string (nullable = true)
 |-- Subscribers_nov: integer (nullable = true)
 |-- Country_nov: string (nullable = true)
 |-- Views_avg_nov: integer (nullable = true)

root
 |-- Channel_name_dic: string (nullable = true)
 |-- Youtuber_name_dic: string (nullable = true)
 |-- Category_dic: string (nullable = true)
 |

In [75]:
df_jun = df_youtube_jun.select(
    "Channel_name_jun", "Youtuber_name_jun", "Subscribers_jun", "Views_avg_jun"
)

df_sep = df_youtube_sep.select(
    "Channel_name_sep", "Youtuber_name_sep", "Subscribers_sep", "Views_avg_sep"
)

df_nov = df_youtube_nov.select(
    "Channel_name_nov", "Youtuber_name_nov", "Subscribers_nov", "Views_avg_nov"
)

df_dic = df_youtube_dic.select(
    "Channel_name_dic", "Youtuber_name_dic", "Subscribers_dic", "Views_avg_dic"
)

In [78]:
df_jun.show()

+--------------------+--------------------+---------------+-------------+
|        Channel_name|       Youtuber_name|Subscribers_jun|Views_avg_jun|
+--------------------+--------------------+---------------+-------------+
|             juptube|  Galinha Pintadinha|       20150883|     20150883|
|baby big mouth ki...|Baby Big Mouth Ki...|       47559011|     47559011|
|     shfa show india|     shfa show India|        2225335|      2225335|
|blippi - educatio...|Blippi - Educatio...|       42738863|     42738863|
|      infobellstamil|   infobells - Tamil|       47403732|     47403732|
|  diana and roma ara|  Diana and Roma ARA|       45898354|     45898354|
|peppalatinoamericano|Peppa Pig Español...|       15243388|     15243388|
|fun for kids tv -...|Fun For Kids TV -...|       33830786|     33830786|
|    supersimplesongs|Super Simple Song...|        1174862|      1174862|
|  diana and roma esp|  Diana and Roma ESP|        6613868|      6613868|
|          d billions|          D Bill

In [77]:
df_jun = df_jun.withColumnRenamed("Channel_name_jun", "Channel_name") \
               .withColumnRenamed("Youtuber_name_jun", "Youtuber_name") \
               .withColumnRenamed("Subscribers_jun", "Subscribers_jun") \
               .withColumnRenamed("Views_avg_jun", "Views_avg_jun")

df_sep = df_sep.withColumnRenamed("Channel_name_sep", "Channel_name") \
               .withColumnRenamed("Youtuber_name_sep", "Youtuber_name") \
               .withColumnRenamed("Subscribers_sep", "Subscribers_sep") \
               .withColumnRenamed("Views_avg_sep", "Views_avg_sep")

df_nov = df_nov.withColumnRenamed("Channel_name_nov", "Channel_name") \
               .withColumnRenamed("Youtuber_name_nov", "Youtuber_name") \
               .withColumnRenamed("Subscribers_nov", "Subscribers_nov") \
               .withColumnRenamed("Views_avg_nov", "Views_avg_nov")

df_dic = df_dic.withColumnRenamed("Channel_name_dic", "Channel_name") \
               .withColumnRenamed("Youtuber_name_dic", "Youtuber_name") \
               .withColumnRenamed("Subscribers_dic", "Subscribers_dic") \
               .withColumnRenamed("Views_avg_dic", "Views_avg_dic")

In [79]:
df_combined = df_jun.join(df_sep, on=["Channel_name", "Youtuber_name"], how="outer") \
                   .join(df_nov, on=["Channel_name", "Youtuber_name"], how="outer") \
                   .join(df_dic, on=["Channel_name", "Youtuber_name"], how="outer")

In [80]:
df_combined.show()

+--------------------+--------------------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+
|        Channel_name|             Youtuber_name|Subscribers_jun|Views_avg_jun|Subscribers_sep|Views_avg_sep|Subscribers_nov|Views_avg_nov|Subscribers_dic|Views_avg_dic|
+--------------------+--------------------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+
|                NULL|                      NULL|           NULL|         NULL|       23948975|     23948975|           NULL|         NULL|           NULL|         NULL|
|     #refugio mental|           #Refugio Mental|        8659618|      8659618|       13800775|     13800775|       39491436|     39491436|       49327512|     49327512|
|     #refúgio mental|           #Refúgio Mental|       37966812|     37966812|       18697685|     18697685|        5172010|      5172010|       4614

In [82]:
# Crecimiento en Subscribers entre junio y septiembre
df_combined = df_combined.withColumn("growth_subscribers_jun_sep", 
                                     (col("Subscribers_sep") - col("Subscribers_jun")) / col("Subscribers_jun") * 100)

# Crecimiento en Views_avg entre junio y septiembre
df_combined = df_combined.withColumn("growth_views_jun_sep", 
                                     (col("Views_avg_sep") - col("Views_avg_jun")) / col("Views_avg_jun") * 100)

# Crecimiento en Subscribers entre septiembre y noviembre
df_combined = df_combined.withColumn("growth_subscribers_sep_nov", 
                                     (col("Subscribers_nov") - col("Subscribers_sep")) / col("Subscribers_sep") * 100)

# Crecimiento en Views_avg entre septiembre y noviembre
df_combined = df_combined.withColumn("growth_views_sep_nov", 
                                     (col("Views_avg_nov") - col("Views_avg_sep")) / col("Views_avg_sep") * 100)

# Crecimiento en Subscribers entre noviembre y diciembre
df_combined = df_combined.withColumn("growth_subscribers_nov_dic", 
                                     (col("Subscribers_dic") - col("Subscribers_nov")) / col("Subscribers_nov") * 100)

# Crecimiento en Views_avg entre noviembre y diciembre
df_combined = df_combined.withColumn("growth_views_nov_dic", 
                                     (col("Views_avg_dic") - col("Views_avg_nov")) / col("Views_avg_nov") * 100)

In [83]:
df_combined = df_combined.fillna(0, subset=["growth_subscribers_jun_sep", "growth_views_jun_sep",
                                            "growth_subscribers_sep_nov", "growth_views_sep_nov",
                                            "growth_subscribers_nov_dic", "growth_views_nov_dic"])

In [84]:
df_combined.show(truncate=False)

+-----------------------+--------------------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+
|Channel_name           |Youtuber_name             |Subscribers_jun|Views_avg_jun|Subscribers_sep|Views_avg_sep|Subscribers_nov|Views_avg_nov|Subscribers_dic|Views_avg_dic|growth_subscribers_jun_sep|growth_views_jun_sep|growth_subscribers_sep_nov|growth_views_sep_nov|growth_subscribers_nov_dic|growth_views_nov_dic|
+-----------------------+--------------------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+
|NULL                   |NULL                    

In [85]:

# Calcular el crecimiento promedio de Subscribers entre los meses
df_combined = df_combined.withColumn(
    "growth_subscribers_total",
    (col("growth_subscribers_jun_sep") + col("growth_subscribers_sep_nov") + col("growth_subscribers_nov_dic")) / 3
)

# Calcular el crecimiento promedio de Views_avg entre los meses
df_combined = df_combined.withColumn(
    "growth_views_total",
    (col("growth_views_jun_sep") + col("growth_views_sep_nov") + col("growth_views_nov_dic")) / 3
)

# Calcular el crecimiento total combinado, ponderando igual entre Subscribers y Views_avg
df_combined = df_combined.withColumn(
    "growth_total",
    (col("growth_subscribers_total") + col("growth_views_total")) / 2
)

In [86]:
# Ordenar por el crecimiento total y seleccionar el top 10
df_top_10 = df_combined.orderBy(col("growth_total").desc()).limit(10)

In [87]:
df_top_10.select("Channel_name", "Youtuber_name", "growth_total").show(truncate=False)

+-------------------------+-------------------------+------------------+
|Channel_name             |Youtuber_name            |growth_total      |
+-------------------------+-------------------------+------------------+
|5-minute crafts          |5-Minute Crafts          |1777.4974514442147|
|antrax                   |Antrax                   |1726.3457718606085|
|kompastvnews             |KOMPASTV                 |1444.6943092399167|
|wiederdude               |WiederDude               |1444.219759630622 |
|zach choi asmr           |Zach Choi ASMR           |1218.8354007872351|
|vlad and niki esp        |Vlad and Niki ESP        |1174.8986660380235|
|heidi y zidane en español|Heidi y Zidane en español|1138.7132540141833|
|oum walid                |Oum Walid                |1098.5893670226699|
|caso cerrado             |Caso Cerrado             |956.5576302236442 |
|dyland pros              |Dyland PROS              |922.0356637917907 |
+-------------------------+------------------------

In [88]:
# Crecimiento porcentual de Subscribers entre los meses
df_combined = df_combined.withColumn(
    "growth_subscribers_jun_sep",
    ((col("Subscribers_sep") - col("Subscribers_jun")) / col("Subscribers_jun")) * 100
)

df_combined = df_combined.withColumn(
    "growth_subscribers_sep_nov",
    ((col("Subscribers_nov") - col("Subscribers_sep")) / col("Subscribers_sep")) * 100
)

df_combined = df_combined.withColumn(
    "growth_subscribers_nov_dic",
    ((col("Subscribers_dic") - col("Subscribers_nov")) / col("Subscribers_nov")) * 100
)

# Crecimiento porcentual de Views_avg entre los meses
df_combined = df_combined.withColumn(
    "growth_views_jun_sep",
    ((col("Views_avg_sep") - col("Views_avg_jun")) / col("Views_avg_jun")) * 100
)

df_combined = df_combined.withColumn(
    "growth_views_sep_nov",
    ((col("Views_avg_nov") - col("Views_avg_sep")) / col("Views_avg_sep")) * 100
)

df_combined = df_combined.withColumn(
    "growth_views_nov_dic",
    ((col("Views_avg_dic") - col("Views_avg_nov")) / col("Views_avg_nov")) * 100
)

In [89]:
# Promedio del crecimiento porcentual de Subscribers
df_combined = df_combined.withColumn(
    "growth_subscribers_total",
    (col("growth_subscribers_jun_sep") + col("growth_subscribers_sep_nov") + col("growth_subscribers_nov_dic")) / 3
)

# Promedio del crecimiento porcentual de Views_avg
df_combined = df_combined.withColumn(
    "growth_views_total",
    (col("growth_views_jun_sep") + col("growth_views_sep_nov") + col("growth_views_nov_dic")) / 3
)

# Crecimiento porcentual total combinado
df_combined = df_combined.withColumn(
    "growth_total",
    (col("growth_subscribers_total") + col("growth_views_total")) / 2
)

In [93]:
# Ordenar por el crecimiento total y seleccionar el top 10
df_top_10 = df_combined.orderBy(col("growth_total").desc()).limit(10)

# Mostrar el resultado
df_top_10.select("Channel_name", "Youtuber_name", "growth_total", "growth_views_total").show(truncate=False)

+-----------------------------------------------+-----------------------------------------------+------------------+------------------+
|Channel_name                                   |Youtuber_name                                  |growth_total      |growth_views_total|
+-----------------------------------------------+-----------------------------------------------+------------------+------------------+
|kids tv - nursery rhymes and baby songs        |Kids TV - Nursery Rhymes And Baby Songs        |1239.9121267760468|1239.9121267760468|
|hasanjr11                                      |hasanjr11                                      |1188.5757529660323|1188.5757529660323|
|playstation                                    |PlayStation                                    |1157.7188848153767|1157.7188848153767|
|lulu99                                         |Lulu99                                         |1131.8284696885378|1131.8284696885378|
|#refúgio mental                                